In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
from exobuilder.contracts.futureschain import FuturesChain
from exobuilder.contracts.futurecontract import FutureContract
from exobuilder.tests.assetindexdict import AssetIndexDicts
from datetime import datetime, date, timedelta, time as dttime
from exobuilder.contracts.instrument import Instrument
from exobuilder.data.datasource_mongo import DataSourceMongo
from exobuilder.data.datasource_sql import DataSourceSQL
from exobuilder.data.assetindex_mongo import AssetIndexMongo
from exobuilder.data.exostorage import EXOStorage
from exobuilder.exo.exoenginebase import ExoEngineBase
from exobuilder.exo.transaction import Transaction
import time
from exobuilder.algorithms.rollover_helper import RolloverHelper
from exobuilder.smartexo.utils import SmartEXOUtils

import importlib
import logging
importlib.reload(logging);
import matplotlib.pyplot as plt

In [ ]:
SMARTUTILS_CONFIG = {
    'futures_limit': 3,           # How many futures expirations to load 
    'options_limit': 20,          # Increase this value if you are getting too many warnings about absent strikes
    'verbosive_logging': False    # Enable extra debug information
}

EXO_CONFIG = {
    # Pick existing directory for log file
    'log_file_path': ''          # Disable logging
    #'log_file_path': '~/'       # Linux home dir
    #'log_file_path': 'C:\'      # Windows C:\
}

# TODO: select start date for SmartEXO calculation
base_date = datetime(2015, 1, 1, 12, 45, 0)

# Define SmartEXO class

#### TODO:
1. Change EXO_NAME property inside class to reflect EXO features
2. Change ASSET_LIST to select pool of products
3. Change regime picking algorithm parameters in '__def init__' to customize SmartEXO regime switching
4. Change new_position_bullish_zone() / new_position_bearish_zone() / new_position_neutral_zone / manage_opened_position to customize position management

**Warning**: please do not change SmartEXOGeneric inside the notebook it could be done after deployment

In [ ]:
from exobuilder.smartexo.smartexo_ichi import SmartEXOIchi

class SmartEXOGeneric(SmartEXOIchi):
    
    # Change the EXO name to reflect SmartEXO behavior
    EXO_NAME = 'SmartEXO_Bullish_Ichi__class_based_test'
    
    # select instruments list for SMART EXO calculation
    # ASSET_LIST  = ['CL', 'ES', 'NG', 'ZC', 'ZS', 'ZW', 'ZN']    
    ASSET_LIST = ['ES']
    
    def __init__(self, symbol, direction, date, datasource, **kwargs):        
        super().__init__(symbol, direction, date, datasource,
                         #
                         # Change following values if you need to customize Ichimoku settings
                         #
                         conversion_line_period = 9,
                         base_line_period=26,
                         leading_spans_lookahead_period=52,
                         leading_span_b_period=52
                        )
    
    @staticmethod  
    def new_position_bullish_zone(date, fut, opt_chain):
        """
        opt_chain.get_by_delta(delta_value) help:

        Search option contract by delta value:
        If delta ==  0.5 - returns ATM call
        If delta == -0.5 - returns ATM put

        If delta > 0.5 - returns ITM call near target delta
        If delta < -0.5 - returns ITM put near target delta

        If delta > 0 and < 0.5 - returns OTM call
        If delta < 0 and > -0.5 - returns OTM put

        If delta <= -1 or >= 1 or 0 - raises error

        Examples:
        # ATM put (delta = -0.5)
        Transaction(opt_chain.get_by_delta(-0.5), date, 1.0),
        # OTM put (delta = -0.25)
        Transaction(opt_chain.get_by_delta(-0.25), date, 1.0),
        # ITM put (delta = -0.75)
        Transaction(opt_chain.get_by_delta(-0.75), date, 1.0),

        # ATM call (delta = 0.5)
        Transaction(opt_chain.get_by_delta(0.5), date, 1.0),
        # OTM call (delta = 0.25)
        Transaction(opt_chain.get_by_delta(0.25), date, 1.0),
        # ITM call (delta = 0.75)
        Transaction(opt_chain.get_by_delta(0.75), date, 1.0),
        """
        
        # Edit transactions to trade
        trans_list = [
                #Transaction(asset, date, qty, price=[MktPrice], leg_name=['' or unique name])
                #
                #
                Transaction(opt_chain.get_by_delta(0.15), date, 1.0),
                Transaction(opt_chain.get_by_delta(-0.25), date, -1.0),
                Transaction(opt_chain.get_by_delta(0.25), date, 0.0),
                Transaction(opt_chain.get_by_delta(0.6), date, -0.0),
                Transaction(opt_chain.get_by_delta(0.20), date, 0.0),
                Transaction(opt_chain.get_by_delta(0.05), date, 0.0),
                ]
        return trans_list
    
    @staticmethod
    def new_position_bearish_zone(date, fut, opt_chain):
        trans_list = [
                #Transaction(asset, date, qty, price=[MktPrice], leg_name=['' or unique name])
                #
                #
                Transaction(opt_chain.get_by_delta(0.15), date, 1.0),
                Transaction(opt_chain.get_by_delta(-0.25), date, -1.0),
                Transaction(opt_chain.get_by_delta(0.25), date, 0.0),
                Transaction(opt_chain.get_by_delta(0.6), date, -0.0),
                Transaction(opt_chain.get_by_delta(0.20), date, 0.0),
                Transaction(opt_chain.get_by_delta(0.05), date, 0.0),
                
                ]

        return trans_list
    
    @staticmethod
    def new_position_neutral_zone(date, fut, opt_chain):
        # Edit transactions to trade
        trans_list = [
                    #Transaction(asset, date, qty, price=[MktPrice], leg_name=['' or unique name])
                    #
                    #
                    Transaction(opt_chain.get_by_delta(0.15), date, 1.0),
                    Transaction(opt_chain.get_by_delta(-0.25), date, -1.0),
                    Transaction(opt_chain.get_by_delta(-0.6), date, 0.0),
                    Transaction(opt_chain.get_by_delta(0.6), date, 0.0),
                    Transaction(opt_chain.get_by_delta(0.20), date, 0.0),
                    Transaction(opt_chain.get_by_delta(0.05), date, 0.0),           
                    ]
        return trans_list
    

    def manage_opened_position(self, date, fut, opt_chain, regime, opened_position):
        logging.debug('Current position delta: {0}'.format(opened_position.delta))   
    
        delta = opened_position.delta

        trans_list = []

        if regime == 1:
            # Delta bounds checks for BULLISH regime        
            # Check required delta bounds values for position 
            if delta < 0.25 or delta > 0.75:
                # Do not change next
                logging.debug('Rebalancing bullish position')   
                trans_list += opened_position.close_all_translist()
                trans_list += self.new_position_bullish_zone(date, fut, opt_chain)
                return trans_list
        if regime == -1:
            # Delta bounds checks for BEARISH regime        
            # Check required delta bounds values for position 
            if delta < -0.75 or delta > -0.25:
                # Do not change next
                logging.debug('Rebalancing bearish position')   
                trans_list += opened_position.close_all_translist()
                trans_list += self.new_position_bearish_zone(date, fut, opt_chain)
                return trans_list
        if regime == 0:          
            # Delta bounds checks for NEUTRAL regime        
            # Check required delta bounds values for position 
            if delta < -0.25 or delta > 0.25:
                # Do not change next
                logging.debug('Rebalancing neutral position')   
                trans_list += opened_position.close_all_translist()
                trans_list += self.new_position_neutral_zone(date, fut, opt_chain)
                return trans_list         


In [ ]:
smart_utils = SmartEXOUtils(SmartEXOGeneric, **SMARTUTILS_CONFIG)

# WhatIF analysis

In [ ]:
analysis_date = datetime(2015, 1, 2, 23, 59)
analysis_instrument = "CL"

WHATIF_CONFIG = {
    'strikes_on_graph': 80,

    # What if scenarios
    'whatif_iv_change': 0.0,               # Change in IV level ( -0.01 - means IV drop in 1% )
    'whatif_days_to_expiration': None,     # Custom days to expiration, if None - use current (i.e. ignored)
}

figsize(10, 5)

### Bullish zone position

In [ ]:
smart_utils.plot_transactions_payoff(SmartEXOGeneric.new_position_bullish_zone, analysis_date, analysis_instrument, **WHATIF_CONFIG )

### Bearish zone position

In [ ]:
smart_utils.plot_transactions_payoff(SmartEXOGeneric.new_position_bearish_zone, analysis_date, analysis_instrument, **WHATIF_CONFIG )

### Neutral zone position

In [ ]:
smart_utils.plot_transactions_payoff(SmartEXOGeneric.new_position_neutral_zone, analysis_date, analysis_instrument, **WHATIF_CONFIG )

# SmartEXO building

In [ ]:
smart_utils.build_smartexo(base_date, **EXO_CONFIG)

In [ ]:
smart_utils.plot_smartexo_price()

# Deployment

Uncomment next line to clear temporary SmartEXO data from DB, and start EXO backfilling process from beginning.

In [ ]:
#smart_utils.clear_smartexo()